# Reinforcement Learning Fundamentals: From Theory to Practice

## Table of Contents
1. [Introduction to Reinforcement Learning](#intro)
2. [Exploration vs Exploitation](#exploration)
3. [Markov Decision Processes (MDPs)](#mdp)
4. [Value Functions vs Policies](#value-policy)
5. [Value Function Approximation](#approximation)
6. [Deep Q-Network Implementation for MountainCar](#dqn)
7. [Training Visualization and Analysis](#visualization)

---

This notebook provides a comprehensive introduction to Reinforcement Learning concepts with practical implementations.

## Setup and Imports

First, let's import all necessary libraries and set up our environment.

In [ ]:
# Azure ML Compatible Setup
# First, let's fix the numpy compatibility issue
import sys
import subprocess

# Check current environment
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")

# Fix numpy compatibility for Azure ML
try:
    # Uninstall and reinstall packages in correct order
    print("Fixing numpy compatibility...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall", "numpy==1.23.5"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall", "pandas==1.5.3"])
    print("Numpy and pandas reinstalled successfully")
except Exception as e:
    print(f"Warning: Could not reinstall packages: {e}")
    print("Continuing with existing packages...")

# Now import libraries
import numpy as np
import matplotlib.pyplot as plt
from collections import deque, namedtuple
import random
import time
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

# Try to import seaborn, but continue without it if it fails
try:
    import seaborn as sns
    sns.set_palette("husl")
    SEABORN_AVAILABLE = True
except ImportError as e:
    print(f"Warning: Seaborn not available ({e}). Using matplotlib only.")
    SEABORN_AVAILABLE = False

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# OpenAI Gym
import gym

# Visualization settings
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('ggplot')
        print("Using ggplot style instead of seaborn")

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")
print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"Gym version: {gym.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")

## <a id='intro'></a>1. Introduction to Reinforcement Learning

Reinforcement Learning (RL) is a paradigm of machine learning where an **agent** learns to make decisions by interacting with an **environment**. The agent receives **rewards** or **penalties** based on its actions, and its goal is to learn a strategy (policy) that maximizes cumulative reward over time.

### Key Components:
- **Agent**: The learner and decision maker
- **Environment**: Everything the agent interacts with
- **State (s)**: The current situation of the agent
- **Action (a)**: What the agent can do
- **Reward (r)**: Feedback from the environment
- **Policy (π)**: The agent's strategy for choosing actions
- **Value Function (V)**: Expected future reward from a state

## <a id='exploration'></a>2. Exploration vs Exploitation

One of the fundamental challenges in RL is the **exploration-exploitation dilemma**:
- **Exploitation**: Choose the best-known action to maximize immediate reward
- **Exploration**: Try new actions to discover potentially better long-term strategies

Let's demonstrate this with a simple multi-armed bandit problem:

In [ ]:
class MultiArmedBandit:
    """Simple multi-armed bandit environment for demonstrating exploration vs exploitation."""
    
    def __init__(self, n_arms=10):
        self.n_arms = n_arms
        # True reward probabilities (unknown to agent)
        self.true_probs = np.random.beta(2, 2, n_arms)
        
    def pull(self, arm):
        """Pull an arm and receive a reward."""
        return 1 if np.random.random() < self.true_probs[arm] else 0
    
    def get_optimal_arm(self):
        """Return the index of the best arm."""
        return np.argmax(self.true_probs)


class EpsilonGreedyAgent:
    """Agent using epsilon-greedy strategy for exploration."""
    
    def __init__(self, n_arms, epsilon=0.1):
        self.n_arms = n_arms
        self.epsilon = epsilon
        self.q_values = np.zeros(n_arms)  # Estimated values
        self.n_pulls = np.zeros(n_arms)   # Number of times each arm was pulled
        
    def select_arm(self):
        """Select an arm using epsilon-greedy strategy."""
        if np.random.random() < self.epsilon:
            # Exploration: choose random arm
            return np.random.randint(self.n_arms)
        else:
            # Exploitation: choose best known arm
            return np.argmax(self.q_values)
    
    def update(self, arm, reward):
        """Update Q-value estimates."""
        self.n_pulls[arm] += 1
        # Incremental update rule
        self.q_values[arm] += (reward - self.q_values[arm]) / self.n_pulls[arm]


# Experiment: Compare different epsilon values
def run_bandit_experiment(epsilon_values, n_steps=1000, n_runs=100):
    results = {eps: [] for eps in epsilon_values}
    
    for eps in epsilon_values:
        cumulative_regret = np.zeros(n_steps)
        
        for run in range(n_runs):
            bandit = MultiArmedBandit()
            agent = EpsilonGreedyAgent(bandit.n_arms, epsilon=eps)
            optimal_arm = bandit.get_optimal_arm()
            
            regret = 0
            for step in range(n_steps):
                arm = agent.select_arm()
                reward = bandit.pull(arm)
                agent.update(arm, reward)
                
                # Calculate regret (difference from optimal)
                optimal_reward = bandit.true_probs[optimal_arm]
                actual_reward = bandit.true_probs[arm]
                regret += optimal_reward - actual_reward
                cumulative_regret[step] += regret
        
        results[eps] = cumulative_regret / n_runs
    
    return results


# Run experiment
epsilon_values = [0.0, 0.01, 0.1, 0.3]
results = run_bandit_experiment(epsilon_values)

# Visualize results
plt.figure(figsize=(12, 6))

for eps, regret in results.items():
    plt.plot(regret, label=f'ε = {eps}', linewidth=2)

plt.xlabel('Steps', fontsize=12)
plt.ylabel('Cumulative Regret', fontsize=12)
plt.title('Exploration vs Exploitation: Effect of Epsilon on Cumulative Regret', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Analysis:")
print("- ε = 0.0 (pure exploitation): Gets stuck with suboptimal arms")
print("- ε = 0.01: Balances exploration and exploitation well")
print("- ε = 0.1: Good balance for this problem")
print("- ε = 0.3: Too much exploration, slower convergence")

## <a id='mdp'></a>3. Markov Decision Processes (MDPs)

An MDP is a mathematical framework for modeling decision-making problems. It consists of:
- **S**: Set of states
- **A**: Set of actions
- **P**: State transition probability matrix P(s'|s,a)
- **R**: Reward function R(s,a,s')
- **γ**: Discount factor (0 ≤ γ ≤ 1)

The **Markov property** states that the future depends only on the current state, not the history.

Let's create a simple grid world MDP:

In [ ]:
class GridWorldMDP:
    """Simple grid world environment to demonstrate MDP concepts."""
    
    def __init__(self, size=5):
        self.size = size
        self.n_states = size * size
        self.n_actions = 4  # Up, Down, Left, Right
        
        # Define special states
        self.goal_state = (size-1, size-1)  # Bottom-right corner
        self.trap_states = [(1, 1), (2, 3), (3, 1)]  # Trap states
        
        # Action mappings
        self.actions = {
            0: (-1, 0),  # Up
            1: (1, 0),   # Down
            2: (0, -1),  # Left
            3: (0, 1)    # Right
        }
        
        # Initialize transition probabilities and rewards
        self._init_dynamics()
    
    def _init_dynamics(self):
        """Initialize transition probabilities and rewards."""
        self.transitions = {}
        self.rewards = {}
        
        for i in range(self.size):
            for j in range(self.size):
                state = (i, j)
                
                for action in range(self.n_actions):
                    # Deterministic transitions (can be made stochastic)
                    next_state = self._get_next_state(state, action)
                    self.transitions[(state, action)] = next_state
                    
                    # Define rewards
                    if next_state == self.goal_state:
                        self.rewards[(state, action, next_state)] = 10.0
                    elif next_state in self.trap_states:
                        self.rewards[(state, action, next_state)] = -5.0
                    else:
                        self.rewards[(state, action, next_state)] = -0.1
    
    def _get_next_state(self, state, action):
        """Get next state given current state and action."""
        di, dj = self.actions[action]
        next_i = max(0, min(self.size-1, state[0] + di))
        next_j = max(0, min(self.size-1, state[1] + dj))
        return (next_i, next_j)
    
    def step(self, state, action):
        """Take a step in the environment."""
        next_state = self.transitions[(state, action)]
        reward = self.rewards[(state, action, next_state)]
        done = (next_state == self.goal_state)
        return next_state, reward, done
    
    def render(self, state=None, values=None):
        """Visualize the grid world."""
        grid = np.zeros((self.size, self.size))
        
        # Mark special states
        for trap in self.trap_states:
            grid[trap] = -1
        grid[self.goal_state] = 1
        
        if state:
            grid[state] = 0.5
        
        if values is not None:
            # Show value function
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
            
            # Grid visualization
            im1 = ax1.imshow(grid, cmap='RdYlGn', vmin=-1, vmax=1)
            ax1.set_title('Grid World', fontsize=12)
            ax1.set_xticks(range(self.size))
            ax1.set_yticks(range(self.size))
            ax1.grid(True, alpha=0.3)
            
            # Add labels
            for i in range(self.size):
                for j in range(self.size):
                    if (i, j) == self.goal_state:
                        ax1.text(j, i, 'G', ha='center', va='center', fontsize=14, fontweight='bold')
                    elif (i, j) in self.trap_states:
                        ax1.text(j, i, 'T', ha='center', va='center', fontsize=14, fontweight='bold')
            
            # Value function visualization
            value_grid = values.reshape(self.size, self.size)
            im2 = ax2.imshow(value_grid, cmap='coolwarm')
            ax2.set_title('Value Function', fontsize=12)
            ax2.set_xticks(range(self.size))
            ax2.set_yticks(range(self.size))
            ax2.grid(True, alpha=0.3)
            
            # Add value labels
            for i in range(self.size):
                for j in range(self.size):
                    ax2.text(j, i, f'{value_grid[i,j]:.2f}', 
                            ha='center', va='center', fontsize=9)
            
            plt.colorbar(im2, ax=ax2)
            plt.tight_layout()
            plt.show()
        else:
            plt.figure(figsize=(6, 6))
            plt.imshow(grid, cmap='RdYlGn', vmin=-1, vmax=1)
            plt.title('Grid World MDP', fontsize=14)
            plt.colorbar()
            plt.grid(True, alpha=0.3)
            plt.show()


# Create and visualize the MDP
mdp = GridWorldMDP(size=5)
mdp.render()

print("MDP Properties:")
print(f"- State space size: {mdp.n_states}")
print(f"- Action space size: {mdp.n_actions}")
print(f"- Goal state: {mdp.goal_state}")
print(f"- Trap states: {mdp.trap_states}")
print("\nG = Goal (reward: +10)")
print("T = Trap (reward: -5)")
print("Other transitions: -0.1")

## <a id='value-policy'></a>4. Value Functions vs Policies

### Policy (π)
A **policy** π(a|s) defines the agent's behavior by specifying the probability of taking action a in state s.

### Value Function (V)
The **state value function** V^π(s) represents the expected return when starting from state s and following policy π:

$$V^\pi(s) = E_\pi[\sum_{t=0}^{\infty} \gamma^t R_{t+1} | S_0 = s]$$

### Action-Value Function (Q)
The **action-value function** Q^π(s,a) represents the expected return when starting from state s, taking action a, and then following policy π:

$$Q^\pi(s,a) = E_\pi[\sum_{t=0}^{\infty} \gamma^t R_{t+1} | S_0 = s, A_0 = a]$$

Let's implement value iteration and policy iteration:

In [ ]:
class ValueIteration:
    """Value iteration algorithm for solving MDPs."""
    
    def __init__(self, mdp, gamma=0.9, theta=1e-6):
        self.mdp = mdp
        self.gamma = gamma
        self.theta = theta
        self.values = np.zeros(mdp.n_states)
        self.policy = np.zeros((mdp.n_states, mdp.n_actions))
    
    def state_to_index(self, state):
        """Convert 2D state to 1D index."""
        return state[0] * self.mdp.size + state[1]
    
    def index_to_state(self, index):
        """Convert 1D index to 2D state."""
        return (index // self.mdp.size, index % self.mdp.size)
    
    def solve(self, max_iterations=1000):
        """Run value iteration algorithm."""
        iteration_values = []  # Store values for visualization
        
        for iteration in range(max_iterations):
            delta = 0
            new_values = np.copy(self.values)
            
            for state_idx in range(self.mdp.n_states):
                state = self.index_to_state(state_idx)
                
                if state == self.mdp.goal_state:
                    continue
                
                action_values = []
                for action in range(self.mdp.n_actions):
                    next_state, reward, _ = self.mdp.step(state, action)
                    next_idx = self.state_to_index(next_state)
                    action_values.append(reward + self.gamma * self.values[next_idx])
                
                new_values[state_idx] = max(action_values)
                delta = max(delta, abs(new_values[state_idx] - self.values[state_idx]))
            
            self.values = new_values
            iteration_values.append(np.copy(self.values))
            
            if delta < self.theta:
                print(f"Value iteration converged in {iteration + 1} iterations")
                break
        
        # Extract policy from values
        self.extract_policy()
        return iteration_values
    
    def extract_policy(self):
        """Extract greedy policy from value function."""
        for state_idx in range(self.mdp.n_states):
            state = self.index_to_state(state_idx)
            
            if state == self.mdp.goal_state:
                continue
            
            action_values = []
            for action in range(self.mdp.n_actions):
                next_state, reward, _ = self.mdp.step(state, action)
                next_idx = self.state_to_index(next_state)
                action_values.append(reward + self.gamma * self.values[next_idx])
            
            best_action = np.argmax(action_values)
            self.policy[state_idx] = np.zeros(self.mdp.n_actions)
            self.policy[state_idx, best_action] = 1.0
    
    def visualize_policy(self):
        """Visualize the learned policy."""
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # Create grid for visualization
        grid = np.zeros((self.mdp.size, self.mdp.size))
        
        # Mark special states
        for trap in self.mdp.trap_states:
            grid[trap] = -0.5
        grid[self.mdp.goal_state] = 1
        
        ax.imshow(grid, cmap='RdYlGn', alpha=0.3, vmin=-1, vmax=1)
        
        # Draw policy arrows
        arrow_scale = 0.3
        for i in range(self.mdp.size):
            for j in range(self.mdp.size):
                state = (i, j)
                state_idx = self.state_to_index(state)
                
                if state == self.mdp.goal_state:
                    ax.text(j, i, 'GOAL', ha='center', va='center', 
                           fontsize=10, fontweight='bold', color='green')
                    continue
                
                if state in self.mdp.trap_states:
                    ax.text(j, i, 'TRAP', ha='center', va='center', 
                           fontsize=10, fontweight='bold', color='red')
                
                # Get best action
                best_action = np.argmax(self.policy[state_idx])
                
                # Draw arrow
                if best_action == 0:  # Up
                    ax.arrow(j, i, 0, -arrow_scale, head_width=0.1, 
                            head_length=0.1, fc='blue', ec='blue')
                elif best_action == 1:  # Down
                    ax.arrow(j, i, 0, arrow_scale, head_width=0.1, 
                            head_length=0.1, fc='blue', ec='blue')
                elif best_action == 2:  # Left
                    ax.arrow(j, i, -arrow_scale, 0, head_width=0.1, 
                            head_length=0.1, fc='blue', ec='blue')
                elif best_action == 3:  # Right
                    ax.arrow(j, i, arrow_scale, 0, head_width=0.1, 
                            head_length=0.1, fc='blue', ec='blue')
        
        ax.set_title('Learned Policy (Arrows show best action)', fontsize=14)
        ax.set_xticks(range(self.mdp.size))
        ax.set_yticks(range(self.mdp.size))
        ax.grid(True, alpha=0.3)
        ax.set_xlabel('Column', fontsize=12)
        ax.set_ylabel('Row', fontsize=12)
        
        plt.tight_layout()
        plt.show()


# Solve the MDP using value iteration
vi_solver = ValueIteration(mdp, gamma=0.9)
iteration_values = vi_solver.solve()

# Visualize the value function
mdp.render(values=vi_solver.values)

# Visualize the policy
vi_solver.visualize_policy()

# Plot convergence
plt.figure(figsize=(10, 6))
convergence_data = [np.max(np.abs(iteration_values[i] - iteration_values[-1])) 
                   for i in range(len(iteration_values))]
plt.semilogy(convergence_data, linewidth=2)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Max Value Difference from Converged (log scale)', fontsize=12)
plt.title('Value Iteration Convergence', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## <a id='approximation'></a>5. Value Function Approximation

For large or continuous state spaces, we can't store values for every state. Instead, we use **function approximation**:

$$V(s) \approx \hat{V}(s, \theta)$$

Where θ are the parameters of our approximator (e.g., neural network weights).

Common approaches:
- **Linear approximation**: $\hat{V}(s) = \theta^T \phi(s)$
- **Neural networks**: Deep Q-Networks (DQN)
- **Tile coding**: Discretization of continuous spaces

Let's demonstrate linear function approximation:

In [ ]:
class LinearFunctionApproximator:
    """Linear function approximation for value functions."""
    
    def __init__(self, n_features, learning_rate=0.01):
        self.weights = np.random.randn(n_features) * 0.01
        self.learning_rate = learning_rate
    
    def feature_extraction(self, state):
        """Extract features from state (customizable)."""
        # Example: polynomial features for 2D state
        x, y = state
        features = np.array([
            1,  # Bias
            x, y,  # Linear terms
            x**2, y**2,  # Quadratic terms
            x*y,  # Interaction term
            np.exp(-(x**2 + y**2)/10)  # RBF feature
        ])
        return features
    
    def predict(self, state):
        """Predict value for a state."""
        features = self.feature_extraction(state)
        return np.dot(self.weights, features)
    
    def update(self, state, target):
        """Update weights using gradient descent."""
        features = self.feature_extraction(state)
        prediction = self.predict(state)
        error = target - prediction
        
        # Gradient descent update
        self.weights += self.learning_rate * error * features
        
        return error


# Demonstrate function approximation
def demonstrate_approximation():
    """Show how function approximation learns to represent value function."""
    
    # Create a simple 2D value function to approximate
    def true_value_function(x, y):
        return 10 * np.exp(-((x-2)**2 + (y-2)**2)/4) - 5 * np.exp(-((x-3)**2 + (y-1)**2)/2)
    
    # Generate training data
    n_samples = 500
    X = np.random.uniform(0, 5, (n_samples, 2))
    y = [true_value_function(x[0], x[1]) for x in X]
    
    # Train approximator
    approximator = LinearFunctionApproximator(n_features=7, learning_rate=0.01)
    
    errors = []
    for epoch in range(100):
        epoch_error = 0
        for i in range(n_samples):
            error = approximator.update(X[i], y[i])
            epoch_error += error**2
        errors.append(epoch_error / n_samples)
    
    # Visualize results
    fig = plt.figure(figsize=(15, 5))
    
    # Plot 1: True value function
    ax1 = fig.add_subplot(131, projection='3d')
    x_grid = np.linspace(0, 5, 30)
    y_grid = np.linspace(0, 5, 30)
    X_grid, Y_grid = np.meshgrid(x_grid, y_grid)
    Z_true = np.array([[true_value_function(x, y) for x in x_grid] for y in y_grid])
    
    ax1.plot_surface(X_grid, Y_grid, Z_true, cmap='viridis', alpha=0.8)
    ax1.set_title('True Value Function', fontsize=12)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Value')
    
    # Plot 2: Approximated value function
    ax2 = fig.add_subplot(132, projection='3d')
    Z_approx = np.array([[approximator.predict([x, y]) for x in x_grid] for y in y_grid])
    
    ax2.plot_surface(X_grid, Y_grid, Z_approx, cmap='viridis', alpha=0.8)
    ax2.set_title('Approximated Value Function', fontsize=12)
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Value')
    
    # Plot 3: Training error
    ax3 = fig.add_subplot(133)
    ax3.plot(errors, linewidth=2)
    ax3.set_xlabel('Epoch', fontsize=12)
    ax3.set_ylabel('Mean Squared Error', fontsize=12)
    ax3.set_title('Training Error', fontsize=12)
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Final MSE: {errors[-1]:.4f}")
    print(f"Feature weights: {approximator.weights}")


demonstrate_approximation()

## <a id='dqn'></a>6. Deep Q-Network (DQN) for MountainCar

Now let's implement a complete DQN agent to solve the MountainCar-v0 environment. DQN combines:
- Neural network for Q-function approximation
- Experience replay for stable learning
- Target network for stability
- ε-greedy exploration

### MountainCar Environment
- **Goal**: Drive a car up a mountain
- **Challenge**: Car doesn't have enough power to go straight up
- **Strategy**: Must build momentum by going back and forth
- **State**: [position, velocity]
- **Actions**: [push left, no push, push right]

In [ ]:
# Experience replay buffer
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward', 'done'))

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""
    
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, *args):
        """Save a transition."""
        self.buffer.append(Transition(*args))
    
    def sample(self, batch_size):
        """Sample a batch of transitions."""
        return random.sample(self.buffer, batch_size)
    
    def __len__(self):
        return len(self.buffer)


# DQN Network
class DQN(nn.Module):
    """Deep Q-Network architecture."""
    
    def __init__(self, input_dim, output_dim, hidden_dim=128):
        super(DQN, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            nn.init.constant_(module.bias, 0.01)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


# DQN Agent
class DQNAgent:
    """DQN agent for MountainCar."""
    
    def __init__(self, state_dim, action_dim, lr=1e-3, gamma=0.99, 
                 epsilon_start=1.0, epsilon_end=0.01, epsilon_decay=0.995,
                 buffer_size=10000, batch_size=64, tau=0.001):
        
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.tau = tau
        
        # Neural networks
        self.q_network = DQN(state_dim, action_dim).to(device)
        self.target_network = DQN(state_dim, action_dim).to(device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        
        # Optimizer
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        
        # Replay buffer
        self.memory = ReplayBuffer(buffer_size)
        
        # Tracking
        self.losses = []
        self.rewards = []
        self.epsilons = []
    
    def select_action(self, state, training=True):
        """Select action using epsilon-greedy policy."""
        if training and random.random() < self.epsilon:
            return random.randrange(self.action_dim)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            q_values = self.q_network(state_tensor)
            return q_values.argmax(1).item()
    
    def store_transition(self, state, action, next_state, reward, done):
        """Store transition in replay buffer."""
        self.memory.push(state, action, next_state, reward, done)
    
    def update(self):
        """Update Q-network using experience replay."""
        if len(self.memory) < self.batch_size:
            return
        
        # Sample batch
        transitions = self.memory.sample(self.batch_size)
        batch = Transition(*zip(*transitions))
        
        # Convert to tensors
        state_batch = torch.FloatTensor(batch.state).to(device)
        action_batch = torch.LongTensor(batch.action).to(device)
        reward_batch = torch.FloatTensor(batch.reward).to(device)
        next_state_batch = torch.FloatTensor(batch.next_state).to(device)
        done_batch = torch.FloatTensor(batch.done).to(device)
        
        # Current Q values
        current_q_values = self.q_network(state_batch).gather(1, action_batch.unsqueeze(1))
        
        # Next Q values from target network
        with torch.no_grad():
            next_q_values = self.target_network(next_state_batch).max(1)[0]
            target_q_values = reward_batch + (1 - done_batch) * self.gamma * next_q_values
        
        # Compute loss
        loss = F.mse_loss(current_q_values.squeeze(), target_q_values)
        
        # Optimize
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 1.0)
        self.optimizer.step()
        
        # Soft update target network
        self.soft_update_target_network()
        
        self.losses.append(loss.item())
        
        return loss.item()
    
    def soft_update_target_network(self):
        """Soft update of target network parameters."""
        for target_param, param in zip(self.target_network.parameters(), 
                                       self.q_network.parameters()):
            target_param.data.copy_(self.tau * param.data + 
                                   (1.0 - self.tau) * target_param.data)
    
    def decay_epsilon(self):
        """Decay exploration rate."""
        self.epsilon = max(self.epsilon_end, self.epsilon * self.epsilon_decay)
        self.epsilons.append(self.epsilon)


# Training function
def train_dqn_mountaincar(n_episodes=500, max_steps=200, render_interval=100):
    """Train DQN on MountainCar-v0."""
    
    # Create environment
    env = gym.make('MountainCar-v0')
    
    # Get dimensions
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    
    print(f"State dimension: {state_dim}")
    print(f"Action dimension: {action_dim}")
    print(f"Actions: 0=Push left, 1=No push, 2=Push right\n")
    
    # Create agent
    agent = DQNAgent(
        state_dim=state_dim,
        action_dim=action_dim,
        lr=0.001,
        gamma=0.99,
        epsilon_start=1.0,
        epsilon_end=0.01,
        epsilon_decay=0.995,
        buffer_size=10000,
        batch_size=64,
        tau=0.001
    )
    
    # Training metrics
    episode_rewards = []
    episode_lengths = []
    moving_avg_rewards = []
    
    # Training loop
    for episode in range(n_episodes):
        state = env.reset()
        if isinstance(state, tuple):
            state = state[0]
        
        total_reward = 0
        
        for step in range(max_steps):
            # Select and execute action
            action = agent.select_action(state)
            result = env.step(action)
            
            # Handle different gym versions
            if len(result) == 5:
                next_state, reward, terminated, truncated, _ = result
                done = terminated or truncated
            else:
                next_state, reward, done, _ = result
            
            # Custom reward shaping for MountainCar
            position, velocity = next_state
            
            # Reward shaping: encourage reaching the goal
            shaped_reward = reward
            if position >= 0.5:  # Reached goal
                shaped_reward = 10.0
            else:
                # Small penalty for time and bonus for height/velocity
                shaped_reward = -0.1 + position * 0.1 + abs(velocity) * 0.1
            
            # Store transition
            agent.store_transition(state, action, next_state, shaped_reward, done)
            
            # Update network
            loss = agent.update()
            
            total_reward += reward
            state = next_state
            
            if done:
                break
        
        # Decay epsilon
        agent.decay_epsilon()
        
        # Record metrics
        episode_rewards.append(total_reward)
        episode_lengths.append(step + 1)
        
        # Calculate moving average
        if len(episode_rewards) >= 100:
            moving_avg = np.mean(episode_rewards[-100:])
        else:
            moving_avg = np.mean(episode_rewards)
        moving_avg_rewards.append(moving_avg)
        
        # Print progress
        if (episode + 1) % 10 == 0:
            print(f"Episode {episode + 1}/{n_episodes} | "
                  f"Reward: {total_reward:.2f} | "
                  f"Steps: {step + 1} | "
                  f"Epsilon: {agent.epsilon:.3f} | "
                  f"Avg Reward (100 ep): {moving_avg:.2f}")
        
        # Render occasionally
        if (episode + 1) % render_interval == 0 and episode > 0:
            print(f"\n--- Rendering episode {episode + 1} ---")
            test_state = env.reset()
            if isinstance(test_state, tuple):
                test_state = test_state[0]
            
            for _ in range(max_steps):
                env.render()
                action = agent.select_action(test_state, training=False)
                result = env.step(action)
                
                if len(result) == 5:
                    test_state, _, terminated, truncated, _ = result
                    done = terminated or truncated
                else:
                    test_state, _, done, _ = result
                
                if done:
                    break
    
    env.close()
    
    return agent, episode_rewards, episode_lengths, moving_avg_rewards


# Train the agent
print("Training DQN on MountainCar-v0...\n")
agent, rewards, lengths, moving_avg = train_dqn_mountaincar(n_episodes=500)

## <a id='visualization'></a>7. Training Visualization and Analysis

Let's visualize the training process and analyze the agent's performance.

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Episode Rewards
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(rewards, alpha=0.3, color='blue', label='Episode Reward')
ax1.plot(moving_avg, color='red', linewidth=2, label='Moving Average (100 episodes)')
ax1.set_xlabel('Episode', fontsize=11)
ax1.set_ylabel('Total Reward', fontsize=11)
ax1.set_title('Training Progress: Episode Rewards', fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 2. Episode Lengths
ax2 = fig.add_subplot(gs[1, 0])
ax2.plot(lengths, alpha=0.5, color='green')
ax2.set_xlabel('Episode', fontsize=11)
ax2.set_ylabel('Episode Length', fontsize=11)
ax2.set_title('Steps per Episode', fontsize=12)
ax2.grid(True, alpha=0.3)

# 3. Loss curve
ax3 = fig.add_subplot(gs[1, 1])
if len(agent.losses) > 0:
    # Smooth the loss curve
    window_size = min(100, len(agent.losses) // 10)
    if window_size > 0:
        smoothed_losses = np.convolve(agent.losses, 
                                      np.ones(window_size)/window_size, 
                                      mode='valid')
        ax3.plot(smoothed_losses, color='purple', alpha=0.7)
    ax3.set_xlabel('Update Step', fontsize=11)
    ax3.set_ylabel('Loss', fontsize=11)
    ax3.set_title('Training Loss (MSE)', fontsize=12)
    ax3.grid(True, alpha=0.3)

# 4. Epsilon decay
ax4 = fig.add_subplot(gs[1, 2])
ax4.plot(agent.epsilons, color='orange', linewidth=2)
ax4.set_xlabel('Episode', fontsize=11)
ax4.set_ylabel('Epsilon', fontsize=11)
ax4.set_title('Exploration Rate Decay', fontsize=12)
ax4.grid(True, alpha=0.3)

# 5. Reward distribution
ax5 = fig.add_subplot(gs[2, 0])
ax5.hist(rewards, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
ax5.set_xlabel('Total Reward', fontsize=11)
ax5.set_ylabel('Frequency', fontsize=11)
ax5.set_title('Reward Distribution', fontsize=12)
ax5.axvline(np.mean(rewards), color='red', linestyle='--', 
           label=f'Mean: {np.mean(rewards):.2f}')
ax5.legend(fontsize=10)
ax5.grid(True, alpha=0.3)

# 6. Success rate over time
ax6 = fig.add_subplot(gs[2, 1])
window = 50
success_threshold = -110  # MountainCar success if reward > -110
success_rate = []
for i in range(len(rewards)):
    start = max(0, i - window + 1)
    window_rewards = rewards[start:i+1]
    success_rate.append(sum(r > success_threshold for r in window_rewards) / len(window_rewards) * 100)

ax6.plot(success_rate, color='darkgreen', linewidth=2)
ax6.set_xlabel('Episode', fontsize=11)
ax6.set_ylabel('Success Rate (%)', fontsize=11)
ax6.set_title(f'Success Rate (Rolling {window} episodes)', fontsize=12)
ax6.grid(True, alpha=0.3)

# 7. Q-value statistics
ax7 = fig.add_subplot(gs[2, 2])
# Sample some states to check Q-values
test_states = [
    [-1.2, 0.0],  # Starting position
    [-0.5, 0.03],  # Mid position
    [0.3, 0.04],   # Near goal
    [0.5, 0.0]     # Goal position
]
q_values_test = []
with torch.no_grad():
    for state in test_states:
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
        q_vals = agent.q_network(state_tensor).cpu().numpy()[0]
        q_values_test.append(q_vals)

positions = ['Start', 'Mid', 'Near Goal', 'Goal']
x = np.arange(len(positions))
width = 0.25

for i in range(3):
    values = [q[i] for q in q_values_test]
    ax7.bar(x + i*width, values, width, label=f'Action {i}')

ax7.set_xlabel('Position', fontsize=11)
ax7.set_ylabel('Q-value', fontsize=11)
ax7.set_title('Learned Q-values at Different Positions', fontsize=12)
ax7.set_xticks(x + width)
ax7.set_xticklabels(positions)
ax7.legend(fontsize=9)
ax7.grid(True, alpha=0.3)

plt.suptitle('DQN Training Analysis - MountainCar-v0', fontsize=15, fontweight='bold')
plt.show()

# Print summary statistics
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
print(f"Total Episodes: {len(rewards)}")
print(f"Final Epsilon: {agent.epsilon:.4f}")
print(f"\nReward Statistics:")
print(f"  - Mean Reward: {np.mean(rewards):.2f}")
print(f"  - Std Reward: {np.std(rewards):.2f}")
print(f"  - Min Reward: {np.min(rewards):.2f}")
print(f"  - Max Reward: {np.max(rewards):.2f}")
print(f"  - Final 100 Episodes Mean: {np.mean(rewards[-100:]):.2f}")
print(f"\nEpisode Length Statistics:")
print(f"  - Mean Length: {np.mean(lengths):.2f}")
print(f"  - Min Length: {np.min(lengths)}")
print(f"  - Max Length: {np.max(lengths)}")
print(f"\nSuccess Rate (last 100 episodes): {success_rate[-1]:.1f}%")
print(f"Total Network Updates: {len(agent.losses)}")

## Testing the Trained Agent

Let's test our trained agent and visualize its learned policy.

In [ ]:
def test_agent(agent, n_episodes=5, render=False):
    """Test the trained agent."""
    env = gym.make('MountainCar-v0')
    
    test_rewards = []
    test_trajectories = []
    
    for episode in range(n_episodes):
        state = env.reset()
        if isinstance(state, tuple):
            state = state[0]
        
        trajectory = []
        total_reward = 0
        
        for step in range(200):
            if render:
                env.render()
            
            trajectory.append(state.copy())
            
            # Use greedy policy (no exploration)
            action = agent.select_action(state, training=False)
            
            result = env.step(action)
            if len(result) == 5:
                state, reward, terminated, truncated, _ = result
                done = terminated or truncated
            else:
                state, reward, done, _ = result
            
            total_reward += reward
            
            if done:
                trajectory.append(state.copy())
                print(f"Episode {episode + 1}: Reached goal in {step + 1} steps! Total reward: {total_reward:.2f}")
                break
        else:
            print(f"Episode {episode + 1}: Did not reach goal. Total reward: {total_reward:.2f}")
        
        test_rewards.append(total_reward)
        test_trajectories.append(np.array(trajectory))
    
    env.close()
    
    return test_rewards, test_trajectories


# Test the agent
print("Testing trained agent...\n")
test_rewards, test_trajectories = test_agent(agent, n_episodes=5)

# Visualize test trajectories
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, traj in enumerate(test_trajectories[:6]):
    if i < len(axes):
        ax = axes[i]
        positions = traj[:, 0]
        velocities = traj[:, 1]
        
        # Color gradient for time
        colors = plt.cm.viridis(np.linspace(0, 1, len(positions)))
        
        for j in range(len(positions) - 1):
            ax.plot(positions[j:j+2], velocities[j:j+2], 
                   color=colors[j], linewidth=2)
        
        # Mark start and end
        ax.scatter(positions[0], velocities[0], color='green', 
                  s=100, marker='o', label='Start', zorder=5)
        ax.scatter(positions[-1], velocities[-1], color='red', 
                  s=100, marker='*', label='End', zorder=5)
        
        # Mark goal region
        ax.axvline(x=0.5, color='red', linestyle='--', alpha=0.5, label='Goal')
        
        ax.set_xlabel('Position', fontsize=10)
        ax.set_ylabel('Velocity', fontsize=10)
        ax.set_title(f'Episode {i+1} Trajectory', fontsize=11)
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=8)
        ax.set_xlim([-1.3, 0.7])
        ax.set_ylim([-0.08, 0.08])

plt.suptitle('Agent Trajectories in State Space', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\nTest Performance:")
print(f"Average test reward: {np.mean(test_rewards):.2f} ± {np.std(test_rewards):.2f}")
print(f"Success rate: {sum(r > -200 for r in test_rewards) / len(test_rewards) * 100:.1f}%")

## Visualizing the Learned Q-Function

Let's create a heatmap visualization of the learned Q-values across the state space.

In [ ]:
def visualize_q_function(agent, resolution=50):
    """Create heatmap of Q-values across state space."""
    
    # Create grid of states
    positions = np.linspace(-1.2, 0.6, resolution)
    velocities = np.linspace(-0.07, 0.07, resolution)
    
    # Compute Q-values for each state-action pair
    q_values_grid = np.zeros((3, resolution, resolution))
    
    with torch.no_grad():
        for i, pos in enumerate(positions):
            for j, vel in enumerate(velocities):
                state = torch.FloatTensor([pos, vel]).unsqueeze(0).to(device)
                q_vals = agent.q_network(state).cpu().numpy()[0]
                q_values_grid[:, j, i] = q_vals
    
    # Create visualization
    fig, axes = plt.subplots(1, 4, figsize=(18, 4))
    
    action_names = ['Push Left', 'No Push', 'Push Right']
    
    # Plot Q-values for each action
    for idx, (ax, action_name) in enumerate(zip(axes[:3], action_names)):
        im = ax.imshow(q_values_grid[idx], aspect='auto', origin='lower',
                      extent=[-1.2, 0.6, -0.07, 0.07], cmap='coolwarm')
        ax.set_xlabel('Position', fontsize=11)
        ax.set_ylabel('Velocity', fontsize=11)
        ax.set_title(f'Q-values for {action_name}', fontsize=12)
        ax.axvline(x=0.5, color='green', linestyle='--', alpha=0.7, linewidth=2)
        plt.colorbar(im, ax=ax)
    
    # Plot best action
    best_actions = np.argmax(q_values_grid, axis=0)
    im = axes[3].imshow(best_actions, aspect='auto', origin='lower',
                       extent=[-1.2, 0.6, -0.07, 0.07], cmap='viridis',
                       vmin=0, vmax=2)
    axes[3].set_xlabel('Position', fontsize=11)
    axes[3].set_ylabel('Velocity', fontsize=11)
    axes[3].set_title('Best Action (Policy)', fontsize=12)
    axes[3].axvline(x=0.5, color='white', linestyle='--', alpha=0.7, linewidth=2)
    
    # Create custom colorbar for actions
    cbar = plt.colorbar(im, ax=axes[3], ticks=[0, 1, 2])
    cbar.ax.set_yticklabels(['Left', 'None', 'Right'])
    
    plt.suptitle('Learned Q-Function and Policy Visualization', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


visualize_q_function(agent, resolution=40)

## Save the Trained Model

Save the trained model for later use.

In [ ]:
# Save the model
model_path = 'dqn_mountaincar_model.pth'
torch.save({
    'model_state_dict': agent.q_network.state_dict(),
    'optimizer_state_dict': agent.optimizer.state_dict(),
    'epsilon': agent.epsilon,
    'episode_rewards': rewards,
    'episode_lengths': lengths
}, model_path)

print(f"Model saved to {model_path}")
print(f"File size: {os.path.getsize(model_path) / 1024:.2f} KB")

# Example of loading the model
# checkpoint = torch.load(model_path)
# agent.q_network.load_state_dict(checkpoint['model_state_dict'])
# agent.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

## Conclusions and Further Exploration

### What We've Learned

1. **Exploration vs Exploitation**: We demonstrated how ε-greedy exploration balances discovering new strategies with exploiting known good actions.

2. **MDPs**: We implemented a grid world to understand states, actions, transitions, and rewards.

3. **Value Functions vs Policies**: We showed how value iteration can solve MDPs and extract optimal policies.

4. **Function Approximation**: We used neural networks to approximate Q-values for continuous state spaces.

5. **DQN Implementation**: We successfully trained a DQN agent to solve MountainCar using:
   - Experience replay for stable learning
   - Target networks for stability
   - Reward shaping to guide learning

### Key Observations

- The agent learns to build momentum by swinging back and forth
- Q-values show clear action preferences in different state regions
- Training is stable with proper hyperparameters
- Success rate improves significantly over training

### Ideas for Further Exploration

1. **Algorithm Variations**:
   - Implement Double DQN to reduce overestimation
   - Try Dueling DQN architecture
   - Implement Rainbow DQN combining multiple improvements

2. **Different Environments**:
   - CartPole-v1 (simpler, discrete actions)
   - LunarLander-v2 (more complex dynamics)
   - Atari games (high-dimensional observations)

3. **Hyperparameter Tuning**:
   - Learning rate schedules
   - Different network architectures
   - Replay buffer prioritization

4. **Advanced Concepts**:
   - Policy gradient methods (REINFORCE, A2C, PPO)
   - Continuous action spaces with DDPG or SAC
   - Model-based RL approaches

### Resources for Further Learning

- Sutton & Barto: "Reinforcement Learning: An Introduction"
- OpenAI Spinning Up: Educational resource for deep RL
- Stable Baselines3: Production-ready RL algorithms
- DeepMind's RL Course: Comprehensive video lectures

This notebook provides a foundation for understanding and implementing reinforcement learning algorithms. The concepts and code can be extended to tackle more complex problems in robotics, game playing, resource optimization, and many other domains.